In [5]:
############# MNIST 데이터 로딩 및 신경망 정의##########################

# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

# Training settings
batch_size = 64

# MNIST Dataset
# MNIST 데이터 셋을 가져와 train, test 구분
train_dataset = datasets.MNIST(root='./data/',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
test_dataset = datasets.MNIST(root='./data/',
                                train=False,
                                transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
# 각 데이터 셋에서 용도에 맞게 데이터를 랜덤하게 섞어서 Batch size 수에 맞게 가져온다
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)

# 신경망 구조 정의
class Net(nn.Module):
    
    # 초기화
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1_out_np = np.zeros((1, 4, 24, 24))
        self.mp1_out_np = np.zeros((1, 4, 12, 12))
        self.conv2_out_np = np.zeros((1, 12, 8, 8))
        self.mp2_out_np = np.zeros((1, 12, 4, 4))
        self.fc_in_np = np.zeros((1, 192))
        self.fc_out_np = np.zeros((1, 10))
        
        # 커널 크기가 5, 입력 채널이 1, 출력 채널이 3인 CNN 층 구성
        # 출력 채널은 커널의 개수를 의미
        self.conv1 = nn.Conv2d(1, 4, kernel_size=5, bias=False) # Num of weight = 
        
        # 커널 크기가 5, 입력 채널이 3, 출력 채널이 3인 CNN 층 구성
        self.conv2 = nn.Conv2d(4, 12, kernel_size=5, bias=False)
        
        # Max Pooling Layer, 파라미터는 kernel size 의미, 4개의 값 중 최댓값을 출력한다.
        self.mp = nn.MaxPool2d(2)
        
        # Fully Connected Layer
        self.fc_1 = nn.Linear(192, 10) # Num of Weight = 1920
        
    def forward(self, x):
        # 텐서를 일자로 펴기 위해 x의 한 원소의 크기 계산
        in_size = x.size(0)
        
        # CNN 층 출력이 최대 풀링 층을 지나 활성함수 ReLU를 지난다.
        x = self.conv1(x)
        self.conv1_out_np = x.detach().numpy()
        
        x = F.relu(self.mp(x))
        self.mp1_out_np = x.detach().numpy()

        x = self.conv2(x)
        self.conv2_out_np = x.detach().numpy()
        
        x = F.relu(self.mp(x))
        self.mp2_out_np = x.detach().numpy()
        
        # 텐서를 Fully Connected Layer에 넣기 위해 일자로 편다
        x = x.view(in_size, -1) # flatten the tensor
        self.fc_in_np = x.detach().numpy()
        
        # 일자로 편 텐서로 Fully Connected Layer 계산
        x = self.fc_1(x)
        self.fc_out_np = x.detach().numpy()
        
        # 출력층의 활성함수는 Softmax 사용
        return F.log_softmax(x)
    
# Instantiation    
model = Net()

# 최적화 방법으로 Momentum(= 0.5) 방식을 추가한 Stochastical Gradient Descent를 사용한다.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

################### 신경망 학습 및 실행 ##################################

def train(epoch):
    # 해당 모델을 training 상태로 설정하는 함수
    model.train()
    
    # train_loader로 batch_idx, data, target 각각 반복마다 
    # train_loader의 인덱스를 하나씩 늘려가며 data를 넣는다.
    for batch_idx, (data, target) in enumerate(train_loader):
        
        #print(batch_idx)
        
        # Pytorch에서 다룰 수 있게 Variable로 변경
        data, target = Variable(data), Variable(target)
        
        #print(np.shape(data))
        
        # 다음 최적화를 위해 전에 저장되어있던 값을 초기화
        optimizer.zero_grad()
        
        # model을 feed-forward 되어 나온 출력값
        output = model(data)
        
        # Negative Log Likelihood 로 Loss를 계산 
        loss = F.nll_loss(output, target)
        
        # Gradient 값을 계산한다.
        loss.backward()
        
        # back propagation을 통한 가중치값 최적화를 한번 진행
        optimizer.step()
        
        # batch_idx가 10의 배수일 때 => 10번마다
        if batch_idx % 10 == 0:
            #각 값을 모니터 출력
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                # Percentage로 나타내기 위해 100을 곱함
                100. * batch_idx / len(train_loader), loss.item()))
                # data[0] is deprecated

# 테스트에 사용할 함수 정의
def test():
    # 해당 모델을 evaluation 상태로 설정하는 함수
    model.eval()
    # loss 값 stacking 위해 초기화
    test_loss = 0
    # 맞춘 수를 stacking 위해 초기화
    correct = 0
    
    
    for data, target in test_loader:
        
        # 각 배열을 PyTorch로 다룰 수 있는 Variable로 만들어준다
        # volatile was removed and now has no effect
        data, target = Variable(data, volatile=True), Variable(target)
        
        # model을 feed-forward 되어 나온 출력값
        output = model(data)
        
        # sum up batch loss
        #loss 값 stacking
        test_loss += F.nll_loss(output, target, size_average=False).item()
        
        # get the index of the max log-probability
        # max() : 1과 출력 data 중 최대값 출력
        pred = output.data.max(1, keepdim=True)[1]
        
        # view_as(pred) == view(pred.size())
        # => 같은 데이터를 갖는 텐서 배열을 1*n 의 새로운 배열의 텐서로 반환한다.
        # cpu() : CUDA 처리가 불가능하여 CPU로 처리하게 하는 함수
        # sum() : 텐서의 모든 원소 합 반환
        # eq() : 입력받은 두 배열을 비교하여 대응되는 원소값이 갇으면 1,
        # 다르면 0을 갖는 같은 크기의 배열을 반환한다
        # pred와 target 비교하여 둘이 같으면 1을 갖는 배열의 합, 즉 pred 와 target이 맞는 수를 센다 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    # 데이터 셋의 크기로 Loss를 나눠준다    
    test_loss /= len(test_loader.dataset)
    
    # 각 결과값을 모니터 출력
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

C:\Users\user\AppData\Local\Temp\ipykernel_26236\3745148395.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301083
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.304130
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.298058
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.306282
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.302708
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.305286
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.297694
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.296649
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.298526
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.290665
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.283666
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.280283
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.281308
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.281773
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.281706
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.269670
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.273106
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.259615
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.248806
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.231231
Train Epoch: 1 [12800/60000 (

C:\Users\user\AppData\Local\Temp\ipykernel_26236\3745148395.py:153: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.2873, Accuracy: 9054/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.286806
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.224630
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.147867
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.180565
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.112866
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.234496
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.164362
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.184956
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.295385
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.166312
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.190358
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.215772
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.106614
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.199394
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.132229
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.112231
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.366049
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.152716
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.234378
Train Epoch: 2 [12

In [16]:
################# 샘플 통과 후 결과값 추출 #################
from PIL import Image

from fxpmath import Fxp


# 샘플을 이용한 테스트
def sample_test():
    # 해당 모델을 evaluation 상태로 설정하는 함수
    model.eval()
    # loss 값 stacking 위해 초기화
    test_loss = 0
    # 맞춘 수를 stacking 위해 초기화
    correct = 0
    
    
    
    ############## 테스트 샘플 (0 한장) 불러오기 ####################
    
    # target 생성 (label : 0)
    target = Variable(torch.tensor([0]))
    # data 생성 (0_0.bmp)
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (1).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_1.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_1.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (2).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_2.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_2.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (3).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_3.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_3.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (4).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_4.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_4.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (5).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_5.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_5.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (6).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_6.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_6.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (7).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_7.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_7.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (8).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_8.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_8.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (9).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_9.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_9.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (10).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_10.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_10.txt', np_img_re0, fmt='%d',delimiter = " ")
    #print(np_img_re0)
    
    # 0 - 255 => 0 - 1 로 정규화, np.array => tensor 변환
    data = Variable(torch.tensor((np_img_re0 / 255), dtype = torch.float32))
    
    # model을 feed-forward 되어 나온 출력값
    output = model(data)
    #print(np.shape(output))
    
    # sum up batch loss
    # loss 값 stacking
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    # get the index of the max log-probability
    # max() : 1과 출력 data 중 최대값 출력
    pred = output.data.max(1, keepdim=True)[1]
    
    # view_as(pred) == view(pred.size())
    # => 같은 데이터를 갖는 텐서 배열을 1*n 의 새로운 배열의 텐서로 반환한다.
    # cpu() : CUDA 처리가 불가능하여 CPU로 처리하게 하는 함수
    # sum() : 텐서의 모든 원소 합 반환
    # eq() : 입력받은 두 배열을 비교하여 대응되는 원소값이 갇으면 1,
    # 다르면 0을 갖는 같은 크기의 배열을 반환한다
    # pred와 target 비교하여 둘이 같으면 1을 갖는 배열의 합, 즉 pred 와 target이 맞는 수를 센다 
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    print(correct)
    
    # 데이터 셋의 크기로 Loss를 나눠준다    
    #test_loss /= len(test_loader.dataset)
    
    # 각 결과값을 모니터 출력
    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        #test_loss, correct, len(test_loader.dataset),
        #100. * correct / len(test_loader.dataset)))
    
sample_test()

[[  0   0   0   0   0   0   0   0   3   0   0   3   2   0   0   9   0   8
    8   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   3   7   9   5   0   0   7   1
    0   0   1   5   3   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   6   4   0   0   0   1   3   0   4   0
    3   8   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   4   6   6   4   3   0   0
    4   7   0   1  10   6   0   0   0   0]
 [  0   0   0   0   0   0   0   0   8   2   2   4   0   0   0   0   5   4
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   2   1   7  15   0   6
   12   8  11   6   1   7   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   6   3   0   0   8   3   0   9   7
    0   0   9   0   0   3   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   5   0   0  61 172 247 255 255 166
   44  18  62  37   0   6   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  19  62 227 251 255 24

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [28, 28]

In [10]:
############## Conv1 가중치 값 HEX 추출 ############

# float => int
import numpy as np
int_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0] * 128), dtype = torch.int32)
int_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0] * 128), dtype = torch.int32)
int_conv1_weight_3 =  torch.tensor((model.conv1.weight.data[2][0] * 128), dtype = torch.int32)
int_conv1_weight_4 =  torch.tensor((model.conv1.weight.data[3][0] * 128), dtype = torch.int32)
#int_conv1_bias = torch.tensor((model.conv1.bias.data * 128), dtype = torch.int32)
for i in range(5):
    for j in range(5):
        if int_conv1_weight_1[i][j] > 127:
            int_conv1_weight_1[i][j] = 127
        if int_conv1_weight_2[i][j] > 127:
            int_conv1_weight_2[i][j] = 127
        if int_conv1_weight_3[i][j] > 127:
            int_conv1_weight_3[i][j] = 127
        if int_conv1_weight_4[i][j] > 127:
            int_conv1_weight_4[i][j] = 127
int_conv1_weight_stack = np.vstack((int_conv1_weight_1,int_conv1_weight_2,int_conv1_weight_3,int_conv1_weight_4))
print("Signed")
print(int_conv1_weight_stack)
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_weight_4)
np.savetxt('SW_conv1_weight.txt', int_conv1_weight_stack, fmt='%d',delimiter = " ")
#np.savetxt('conv1_weight_2.txt', int_conv1_weight_2, fmt='%d',delimiter = " ")
#np.savetxt('conv1_weight_3.txt', int_conv1_weight_3, fmt='%d',delimiter = " ")
#np.savetxt('conv1_weight_4.txt', int_conv1_weight_3, fmt='%d',delimiter = " ")
#print(int_conv1_bias)

# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv1_weight_1[i][j] < 0:
            int_conv1_weight_1[i][j] += 256
        if int_conv1_weight_2[i][j] < 0:
            int_conv1_weight_2[i][j] += 256
        if int_conv1_weight_3[i][j] < 0:
            int_conv1_weight_3[i][j] += 256
        if int_conv1_weight_4[i][j] < 0:
            int_conv1_weight_4[i][j] += 256

#for i in range(3):
#    if int_conv1_bias[i] < 0:
#            int_conv1_bias[i] += 256
int_conv1_weight_stack = np.vstack((int_conv1_weight_1,int_conv1_weight_2,int_conv1_weight_3,int_conv1_weight_4))
print ("Unsigned")
print(int_conv1_weight_stack)
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_weight_4)
np.savetxt('HW_conv1_weight.mem', int_conv1_weight_stack, fmt='%1.2X',delimiter = " ")
#print(int_conv1_bias)
#filenames = ['conv1_weight_1.txt', 'conv1_weight_2.txt', 'conv1_weight_3.txt', 'conv1_weight_4.txt']

#with open('conv1_weight.txt', 'w') as outfile:
#    for filename in filenames:
#        with open(filename) as file:
#            for line in file:
#                outfile.write(line)

Signed
[[ 21  47  -9   0  -5]
 [ 91  38  26 -40 -53]
 [127  84  19 -14 -47]
 [127 127  71  49  -4]
 [ 61  90  96  54  35]
 [-21   7 -12  -5   5]
 [-23 -24  19   8   7]
 [ -1   2  -9  25  -7]
 [ -3 -27  30  -1  15]
 [-21 -14  37  -4 -13]
 [-14 -50 -45 -44 -33]
 [-15 -30 -56 -64 -17]
 [ 28 -17 -19   4  54]
 [ 48  65  63  79  44]
 [ 60  69  65  88  32]
 [-10  -5   6  -6 -12]
 [ 14  -4   6 -25   9]
 [ -8 -12 -25   4   8]
 [ -7 -15 -24   4  -3]
 [  0 -15  -1 -16 -21]]
tensor([[ 21,  47,  -9,   0,  -5],
        [ 91,  38,  26, -40, -53],
        [127,  84,  19, -14, -47],
        [127, 127,  71,  49,  -4],
        [ 61,  90,  96,  54,  35]], dtype=torch.int32)
tensor([[-21,   7, -12,  -5,   5],
        [-23, -24,  19,   8,   7],
        [ -1,   2,  -9,  25,  -7],
        [ -3, -27,  30,  -1,  15],
        [-21, -14,  37,  -4, -13]], dtype=torch.int32)
tensor([[-14, -50, -45, -44, -33],
        [-15, -30, -56, -64, -17],
        [ 28, -17, -19,   4,  54],
        [ 48,  65,  63,  79,  44],
  

C:\Users\user\AppData\Local\Temp\ipykernel_26236\1049555460.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0] * 128), dtype = torch.int32)
C:\Users\user\AppData\Local\Temp\ipykernel_26236\1049555460.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0] * 128), dtype = torch.int32)
C:\Users\user\AppData\Local\Temp\ipykernel_26236\1049555460.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv1_weight_3

In [11]:
############## Conv2 가중치 값 HEX 추출 ############
import numpy as np
print(np.shape(model.conv2.weight))

int_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]* 128), dtype = torch.int32)
int_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[0][1]* 128), dtype = torch.int32)
int_conv2_weight_13 =  torch.tensor((model.conv2.weight.data[0][2]* 128), dtype = torch.int32)
int_conv2_weight_14 =  torch.tensor((model.conv2.weight.data[0][3]* 128), dtype = torch.int32)

int_conv2_weight_21 =  torch.tensor((model.conv2.weight.data[1][0] * 128), dtype = torch.int32)
int_conv2_weight_22 =  torch.tensor((model.conv2.weight.data[1][1] * 128), dtype = torch.int32)
int_conv2_weight_23 =  torch.tensor((model.conv2.weight.data[1][2] * 128), dtype = torch.int32)
int_conv2_weight_24 =  torch.tensor((model.conv2.weight.data[1][3] * 128), dtype = torch.int32)

int_conv2_weight_31 =  torch.tensor((model.conv2.weight.data[2][0] * 128), dtype = torch.int32)
int_conv2_weight_32 =  torch.tensor((model.conv2.weight.data[2][1] * 128), dtype = torch.int32)
int_conv2_weight_33 =  torch.tensor((model.conv2.weight.data[2][2] * 128), dtype = torch.int32)
int_conv2_weight_34 =  torch.tensor((model.conv2.weight.data[2][3] * 128), dtype = torch.int32)

int_conv2_weight_41 =  torch.tensor((model.conv2.weight.data[3][0] * 128), dtype = torch.int32)
int_conv2_weight_42 =  torch.tensor((model.conv2.weight.data[3][1] * 128), dtype = torch.int32)
int_conv2_weight_43 =  torch.tensor((model.conv2.weight.data[3][2] * 128), dtype = torch.int32)
int_conv2_weight_44 =  torch.tensor((model.conv2.weight.data[3][3] * 128), dtype = torch.int32)

int_conv2_weight_51 =  torch.tensor((model.conv2.weight.data[4][0] * 128), dtype = torch.int32)
int_conv2_weight_52 =  torch.tensor((model.conv2.weight.data[4][1] * 128), dtype = torch.int32)
int_conv2_weight_53 =  torch.tensor((model.conv2.weight.data[4][2] * 128), dtype = torch.int32)
int_conv2_weight_54 =  torch.tensor((model.conv2.weight.data[4][3] * 128), dtype = torch.int32)

int_conv2_weight_61 =  torch.tensor((model.conv2.weight.data[5][0] * 128), dtype = torch.int32)
int_conv2_weight_62 =  torch.tensor((model.conv2.weight.data[5][1] * 128), dtype = torch.int32)
int_conv2_weight_63 =  torch.tensor((model.conv2.weight.data[5][2] * 128), dtype = torch.int32)
int_conv2_weight_64 =  torch.tensor((model.conv2.weight.data[5][3] * 128), dtype = torch.int32)

int_conv2_weight_71 =  torch.tensor((model.conv2.weight.data[6][0] * 128), dtype = torch.int32)
int_conv2_weight_72 =  torch.tensor((model.conv2.weight.data[6][1] * 128), dtype = torch.int32)
int_conv2_weight_73 =  torch.tensor((model.conv2.weight.data[6][2] * 128), dtype = torch.int32)
int_conv2_weight_74 =  torch.tensor((model.conv2.weight.data[6][3] * 128), dtype = torch.int32)

int_conv2_weight_81 =  torch.tensor((model.conv2.weight.data[7][0] * 128), dtype = torch.int32)
int_conv2_weight_82 =  torch.tensor((model.conv2.weight.data[7][1] * 128), dtype = torch.int32)
int_conv2_weight_83 =  torch.tensor((model.conv2.weight.data[7][2] * 128), dtype = torch.int32)
int_conv2_weight_84 =  torch.tensor((model.conv2.weight.data[7][3] * 128), dtype = torch.int32)

int_conv2_weight_91 =  torch.tensor((model.conv2.weight.data[8][0] * 128), dtype = torch.int32)
int_conv2_weight_92 =  torch.tensor((model.conv2.weight.data[8][1] * 128), dtype = torch.int32)
int_conv2_weight_93 =  torch.tensor((model.conv2.weight.data[8][2] * 128), dtype = torch.int32)
int_conv2_weight_94 =  torch.tensor((model.conv2.weight.data[8][3] * 128), dtype = torch.int32)

int_conv2_weight_101 =  torch.tensor((model.conv2.weight.data[9][0] * 128), dtype = torch.int32)
int_conv2_weight_102 =  torch.tensor((model.conv2.weight.data[9][1] * 128), dtype = torch.int32)
int_conv2_weight_103 =  torch.tensor((model.conv2.weight.data[9][2] * 128), dtype = torch.int32)
int_conv2_weight_104 =  torch.tensor((model.conv2.weight.data[9][3] * 128), dtype = torch.int32)

int_conv2_weight_111 =  torch.tensor((model.conv2.weight.data[10][0] * 128), dtype = torch.int32)
int_conv2_weight_112 =  torch.tensor((model.conv2.weight.data[10][1] * 128), dtype = torch.int32)
int_conv2_weight_113 =  torch.tensor((model.conv2.weight.data[10][2] * 128), dtype = torch.int32)
int_conv2_weight_114 =  torch.tensor((model.conv2.weight.data[10][3] * 128), dtype = torch.int32)

int_conv2_weight_121 =  torch.tensor((model.conv2.weight.data[11][0] * 128), dtype = torch.int32)
int_conv2_weight_122 =  torch.tensor((model.conv2.weight.data[11][1] * 128), dtype = torch.int32)
int_conv2_weight_123 =  torch.tensor((model.conv2.weight.data[11][2] * 128), dtype = torch.int32)
int_conv2_weight_124 =  torch.tensor((model.conv2.weight.data[11][3] * 128), dtype = torch.int32)

for i in range(5):
    for j in range(5):
        if int_conv2_weight_11[i][j] > 127:
            int_conv2_weight_11[i][j] = 127
        if int_conv2_weight_12[i][j] > 127:
            int_conv2_weight_12[i][j] = 127
        if int_conv2_weight_13[i][j] > 127:
            int_conv2_weight_13[i][j] = 127
        if int_conv2_weight_14[i][j] > 127:
            int_conv2_weight_14[i][j] = 127
            
        if int_conv2_weight_21[i][j] > 127:
            int_conv2_weight_21[i][j] = 127
        if int_conv2_weight_22[i][j] > 127:
            int_conv2_weight_22[i][j] = 127
        if int_conv2_weight_23[i][j] > 127:
            int_conv2_weight_23[i][j] = 127
        if int_conv2_weight_24[i][j] > 127:
            int_conv2_weight_24[i][j] = 127   
             
        if int_conv2_weight_31[i][j] > 127:
            int_conv2_weight_31[i][j] = 127
        if int_conv2_weight_32[i][j] > 127:
            int_conv2_weight_32[i][j] = 127
        if int_conv2_weight_33[i][j] > 127:
            int_conv2_weight_33[i][j] = 127
        if int_conv2_weight_34[i][j] > 127:
            int_conv2_weight_34[i][j] = 127   
        
        if int_conv2_weight_41[i][j] > 127:
            int_conv2_weight_41[i][j] = 127
        if int_conv2_weight_42[i][j] > 127:
            int_conv2_weight_42[i][j] = 127
        if int_conv2_weight_43[i][j] > 127:
            int_conv2_weight_43[i][j] = 127
        if int_conv2_weight_44[i][j] > 127:
            int_conv2_weight_44[i][j] = 127
            
        if int_conv2_weight_51[i][j] > 127:
            int_conv2_weight_51[i][j] = 127
        if int_conv2_weight_52[i][j] > 127:
            int_conv2_weight_52[i][j] = 127
        if int_conv2_weight_53[i][j] > 127:
            int_conv2_weight_53[i][j] = 127
        if int_conv2_weight_54[i][j] > 127:
            int_conv2_weight_54[i][j] = 127   
             
        if int_conv2_weight_61[i][j] > 127:
            int_conv2_weight_61[i][j] = 127
        if int_conv2_weight_62[i][j] > 127:
            int_conv2_weight_62[i][j] = 127
        if int_conv2_weight_63[i][j] > 127:
            int_conv2_weight_63[i][j] = 127
        if int_conv2_weight_64[i][j] > 127:
            int_conv2_weight_64[i][j] = 127
        
        if int_conv2_weight_71[i][j] > 127:
            int_conv2_weight_71[i][j] = 127
        if int_conv2_weight_72[i][j] > 127:
            int_conv2_weight_72[i][j] = 127
        if int_conv2_weight_73[i][j] > 127:
            int_conv2_weight_73[i][j] = 127
        if int_conv2_weight_74[i][j] > 127:
            int_conv2_weight_74[i][j] = 127
            
        if int_conv2_weight_81[i][j] > 127:
            int_conv2_weight_81[i][j] = 127
        if int_conv2_weight_82[i][j] > 127:
            int_conv2_weight_82[i][j] = 127
        if int_conv2_weight_83[i][j] > 127:
            int_conv2_weight_83[i][j] = 127
        if int_conv2_weight_84[i][j] > 127:
            int_conv2_weight_84[i][j] = 127   
             
        if int_conv2_weight_91[i][j] > 127:
            int_conv2_weight_91[i][j] = 127
        if int_conv2_weight_92[i][j] > 127:
            int_conv2_weight_92[i][j] = 127
        if int_conv2_weight_93[i][j] > 127:
            int_conv2_weight_93[i][j] = 127
        if int_conv2_weight_94[i][j] > 127:
            int_conv2_weight_94[i][j] = 127  
        
        if int_conv2_weight_101[i][j] > 127:
            int_conv2_weight_101[i][j] = 127
        if int_conv2_weight_102[i][j] > 127:
            int_conv2_weight_102[i][j] = 127
        if int_conv2_weight_103[i][j] > 127:
            int_conv2_weight_103[i][j] = 127
        if int_conv2_weight_104[i][j] > 127:
            int_conv2_weight_104[i][j] = 127
            
        if int_conv2_weight_111[i][j] > 127:
            int_conv2_weight_111[i][j] = 127
        if int_conv2_weight_112[i][j] > 127:
            int_conv2_weight_112[i][j] = 127
        if int_conv2_weight_113[i][j] > 127:
            int_conv2_weight_113[i][j] = 127
        if int_conv2_weight_114[i][j] > 127:
            int_conv2_weight_114[i][j] = 127 
             
        if int_conv2_weight_121[i][j] > 127:
            int_conv2_weight_121[i][j] = 127
        if int_conv2_weight_122[i][j] > 127:
            int_conv2_weight_122[i][j] = 127
        if int_conv2_weight_123[i][j] > 127:
            int_conv2_weight_123[i][j] = 127
        if int_conv2_weight_124[i][j] > 127:
            int_conv2_weight_124[i][j] = 127
            
int_conv2_weight_stack = np.vstack((int_conv2_weight_11,int_conv2_weight_21,int_conv2_weight_31,int_conv2_weight_41,int_conv2_weight_51,int_conv2_weight_61,
                                        int_conv2_weight_71,int_conv2_weight_81,int_conv2_weight_91,int_conv2_weight_101,int_conv2_weight_111,int_conv2_weight_121,
                                        int_conv2_weight_12,int_conv2_weight_22,int_conv2_weight_32,int_conv2_weight_42,int_conv2_weight_52,int_conv2_weight_62,
                                        int_conv2_weight_72,int_conv2_weight_82,int_conv2_weight_92,int_conv2_weight_102,int_conv2_weight_112,int_conv2_weight_122,
                                        int_conv2_weight_13,int_conv2_weight_23,int_conv2_weight_33,int_conv2_weight_43,int_conv2_weight_53,int_conv2_weight_63,
                                        int_conv2_weight_73,int_conv2_weight_83,int_conv2_weight_93,int_conv2_weight_103,int_conv2_weight_113,int_conv2_weight_123,
                                        int_conv2_weight_14,int_conv2_weight_24,int_conv2_weight_34,int_conv2_weight_44,int_conv2_weight_54,int_conv2_weight_64,
                                        int_conv2_weight_74,int_conv2_weight_84,int_conv2_weight_94,int_conv2_weight_104,int_conv2_weight_114,int_conv2_weight_124))
                                        
#int_conv2_weight_stack = np.vstack((int_conv2_weight_11,int_conv2_weight_12,int_conv2_weight_13,int_conv2_weight_14,int_conv2_weight_21,int_conv2_weight_22,int_conv2_weight_23,
                                    #int_conv2_weight_24,int_conv2_weight_31,int_conv2_weight_32,int_conv2_weight_33,int_conv2_weight_34,int_conv2_weight_41,int_conv2_weight_42,
                                    #int_conv2_weight_43,int_conv2_weight_44,int_conv2_weight_51,int_conv2_weight_52,int_conv2_weight_53,int_conv2_weight_54,int_conv2_weight_61,
                                    #int_conv2_weight_62,int_conv2_weight_63,int_conv2_weight_64,int_conv2_weight_71,int_conv2_weight_72,int_conv2_weight_73,int_conv2_weight_74,
                                    #int_conv2_weight_81,int_conv2_weight_82,int_conv2_weight_83,int_conv2_weight_84,int_conv2_weight_91,int_conv2_weight_92,int_conv2_weight_93,
                                    #int_conv2_weight_94,int_conv2_weight_101,int_conv2_weight_102,int_conv2_weight_103,int_conv2_weight_104,int_conv2_weight_111,int_conv2_weight_112,
                                    #int_conv2_weight_113,int_conv2_weight_114,int_conv2_weight_121,int_conv2_weight_122,int_conv2_weight_123,int_conv2_weight_124))
#int_conv2_bias = torch.tensor((model.conv2.bias.data * 128), dtype = torch.int32)

print ("Signed")
print(int_conv2_weight_stack, '\n')
#print(int_conv2_bias)
np.savetxt('SW_conv2_weight.txt', int_conv2_weight_stack, fmt='%d',delimiter = " ")
# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv2_weight_11[i][j] < 0:
            int_conv2_weight_11[i][j] += 256
        if int_conv2_weight_12[i][j] < 0:
            int_conv2_weight_12[i][j] += 256
        if int_conv2_weight_13[i][j] < 0:
            int_conv2_weight_13[i][j] += 256
        if int_conv2_weight_14[i][j] < 0:
            int_conv2_weight_14[i][j] += 256
            
        if int_conv2_weight_21[i][j] < 0:
            int_conv2_weight_21[i][j] += 256
        if int_conv2_weight_22[i][j] < 0:
            int_conv2_weight_22[i][j] += 256
        if int_conv2_weight_23[i][j] < 0:
            int_conv2_weight_23[i][j] += 256
        if int_conv2_weight_24[i][j] < 0:
            int_conv2_weight_24[i][j] += 256   
             
        if int_conv2_weight_31[i][j] < 0:
            int_conv2_weight_31[i][j] += 256
        if int_conv2_weight_32[i][j] < 0:
            int_conv2_weight_32[i][j] += 256
        if int_conv2_weight_33[i][j] < 0:
            int_conv2_weight_33[i][j] += 256
        if int_conv2_weight_34[i][j] < 0:
            int_conv2_weight_34[i][j] += 256   
        
        if int_conv2_weight_41[i][j] < 0:
            int_conv2_weight_41[i][j] += 256
        if int_conv2_weight_42[i][j] < 0:
            int_conv2_weight_42[i][j] += 256
        if int_conv2_weight_43[i][j] < 0:
            int_conv2_weight_43[i][j] += 256
        if int_conv2_weight_44[i][j] < 0:
            int_conv2_weight_44[i][j] += 256
            
        if int_conv2_weight_51[i][j] < 0:
            int_conv2_weight_51[i][j] += 256
        if int_conv2_weight_52[i][j] < 0:
            int_conv2_weight_52[i][j] += 256
        if int_conv2_weight_53[i][j] < 0:
            int_conv2_weight_53[i][j] += 256
        if int_conv2_weight_54[i][j] < 0:
            int_conv2_weight_54[i][j] += 256   
             
        if int_conv2_weight_61[i][j] < 0:
            int_conv2_weight_61[i][j] += 256
        if int_conv2_weight_62[i][j] < 0:
            int_conv2_weight_62[i][j] += 256
        if int_conv2_weight_63[i][j] < 0:
            int_conv2_weight_63[i][j] += 256
        if int_conv2_weight_64[i][j] < 0:
            int_conv2_weight_64[i][j] += 256
        
        if int_conv2_weight_71[i][j] < 0:
            int_conv2_weight_71[i][j] += 256
        if int_conv2_weight_72[i][j] < 0:
            int_conv2_weight_72[i][j] += 256
        if int_conv2_weight_73[i][j] < 0:
            int_conv2_weight_73[i][j] += 256
        if int_conv2_weight_74[i][j] < 0:
            int_conv2_weight_74[i][j] += 256
            
        if int_conv2_weight_81[i][j] < 0:
            int_conv2_weight_81[i][j] += 256
        if int_conv2_weight_82[i][j] < 0:
            int_conv2_weight_82[i][j] += 256
        if int_conv2_weight_83[i][j] < 0:
            int_conv2_weight_83[i][j] += 256
        if int_conv2_weight_84[i][j] < 0:
            int_conv2_weight_84[i][j] += 256   
             
        if int_conv2_weight_91[i][j] < 0:
            int_conv2_weight_91[i][j] += 256
        if int_conv2_weight_92[i][j] < 0:
            int_conv2_weight_92[i][j] += 256
        if int_conv2_weight_93[i][j] < 0:
            int_conv2_weight_93[i][j] += 256
        if int_conv2_weight_94[i][j] < 0:
            int_conv2_weight_94[i][j] += 256   
        
        if int_conv2_weight_101[i][j] < 0:
            int_conv2_weight_101[i][j] += 256
        if int_conv2_weight_102[i][j] < 0:
            int_conv2_weight_102[i][j] += 256
        if int_conv2_weight_103[i][j] < 0:
            int_conv2_weight_103[i][j] += 256
        if int_conv2_weight_104[i][j] < 0:
            int_conv2_weight_104[i][j] += 256
            
        if int_conv2_weight_111[i][j] < 0:
            int_conv2_weight_111[i][j] += 256
        if int_conv2_weight_112[i][j] < 0:
            int_conv2_weight_112[i][j] += 256
        if int_conv2_weight_113[i][j] < 0:
            int_conv2_weight_113[i][j] += 256
        if int_conv2_weight_114[i][j] < 0:
            int_conv2_weight_114[i][j] += 256   
             
        if int_conv2_weight_121[i][j] < 0:
            int_conv2_weight_121[i][j] += 256
        if int_conv2_weight_122[i][j] < 0:
            int_conv2_weight_122[i][j] += 256
        if int_conv2_weight_123[i][j] < 0:
            int_conv2_weight_123[i][j] += 256
        if int_conv2_weight_124[i][j] < 0:
            int_conv2_weight_124[i][j] += 256   
        

int_conv2_weight_stack_ch1 = np.vstack((int_conv2_weight_11,int_conv2_weight_21,int_conv2_weight_31,int_conv2_weight_41,int_conv2_weight_51,int_conv2_weight_61,
                                        int_conv2_weight_71,int_conv2_weight_81,int_conv2_weight_91,int_conv2_weight_101,int_conv2_weight_111,int_conv2_weight_121))
int_conv2_weight_stack_ch2 = np.vstack((int_conv2_weight_12,int_conv2_weight_22,int_conv2_weight_32,int_conv2_weight_42,int_conv2_weight_52,int_conv2_weight_62,
                                        int_conv2_weight_72,int_conv2_weight_82,int_conv2_weight_92,int_conv2_weight_102,int_conv2_weight_112,int_conv2_weight_122))
int_conv2_weight_stack_ch3 = np.vstack((int_conv2_weight_13,int_conv2_weight_23,int_conv2_weight_33,int_conv2_weight_43,int_conv2_weight_53,int_conv2_weight_63,
                                        int_conv2_weight_73,int_conv2_weight_83,int_conv2_weight_93,int_conv2_weight_103,int_conv2_weight_113,int_conv2_weight_123))
int_conv2_weight_stack_ch4 = np.vstack((int_conv2_weight_14,int_conv2_weight_24,int_conv2_weight_34,int_conv2_weight_44,int_conv2_weight_54,int_conv2_weight_64,
                                        int_conv2_weight_74,int_conv2_weight_84,int_conv2_weight_94,int_conv2_weight_104,int_conv2_weight_114,int_conv2_weight_124))
#for i in range(3):
#    if int_conv2_bias[i] < 0:
#            int_conv2_bias[i] += 256

print ("Unsigned")
print(int_conv2_weight_stack_ch1, '\n')
print(int_conv2_weight_stack_ch2, '\n')
print(int_conv2_weight_stack_ch3, '\n')
print(int_conv2_weight_stack_ch4, '\n')

np.savetxt('HW_conv2_weight_ch1.mem', int_conv2_weight_stack_ch1, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_conv2_weight_ch2.mem', int_conv2_weight_stack_ch2, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_conv2_weight_ch3.mem', int_conv2_weight_stack_ch3, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_conv2_weight_ch4.mem', int_conv2_weight_stack_ch4, fmt='%1.2X',delimiter = " ")


#np.savetxt('conv2_bias.mem', int_conv2_bias, fmt='%1.2x',delimiter = " ")


torch.Size([12, 4, 5, 5])
Signed
[[-25 -11   5  18  20]
 [-12  -9 -12  -3  24]
 [ 23  -8 -47 -58 -32]
 ...
 [-12  -9  -5  -4   6]
 [-11  -1  -9   0   9]
 [ -6   5   7  -3   3]] 

Unsigned
[[231 245   5  18  20]
 [244 247 244 253  24]
 [ 23 248 209 198 224]
 [  9  23  20 243 243]
 [254   0  30  33 255]
 [230 216 218 238 228]
 [ 19 251 229 224 214]
 [ 13  16  26  27  11]
 [255   0  11  13  15]
 [  0 254 246 247   6]
 [245 236   7  14  11]
 [218 248  25   6 236]
 [193  26  46 244 223]
 [233  44  50 242 230]
 [ 19  39  26 254 254]
 [249 226 217 229   4]
 [228 237  10  16  21]
 [235   0   9  16 235]
 [  5  17  23 251 252]
 [ 18  33  14  22   8]
 [  2 237 224   0  38]
 [ 15 255 223 238  32]
 [ 31 254 221 238  38]
 [ 23 251 213   0  33]
 [  7 233 245  36  32]
 [  1 251 247 237 221]
 [  3  13 243 241 242]
 [251  19 246 235 254]
 [  1  18  14   0  10]
 [ 20  33  38  38  26]
 [ 11  39  43  23  14]
 [247 235 240 250 255]
 [  2 225 224 221 228]
 [254 253 236 254   3]
 [251 255 255   4   0]
 [255 2

C:\Users\user\AppData\Local\Temp\ipykernel_26236\219235275.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]* 128), dtype = torch.int32)
C:\Users\user\AppData\Local\Temp\ipykernel_26236\219235275.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[0][1]* 128), dtype = torch.int32)
C:\Users\user\AppData\Local\Temp\ipykernel_26236\219235275.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv2_weight_13 =

In [12]:
############## FC Layer가중치 값 HEX 추출 ############

print(np.shape(model.fc_1.weight))
print((model.fc_1.weight * 128).int())

print(np.shape(model.fc_1.bias))
print((model.fc_1.bias * 128 * 256).int())

int_fc_weight = (model.fc_1.weight * 128).int()
int_fc_bias = (model.fc_1.bias * 128 * 256).int()
for i in range(10):
    for j in range(192):
        if int_fc_weight[i][j] > 127 :
            int_fc_weight[i][j] = 127
            
np.savetxt('SW_fc_weight.txt', int_fc_weight, fmt='%d',delimiter = " ")
np.savetxt('SW_fc_bias.txt', int_fc_bias, fmt='%d',delimiter = " ")
# signed int => unsigned int
for i in range(10):
    for j in range(192):
        if int_fc_weight[i][j] < 0 :
            int_fc_weight[i][j] += 256
    if int_fc_bias[i] < 0 :
        int_fc_bias[i] += 256*256
        
print(int_fc_weight)
print(int_fc_bias)

np.savetxt('HW_fc_weight.mem', int_fc_weight, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_fc_bias.mem', int_fc_bias, fmt='%1.2X',delimiter = " ")

torch.Size([10, 192])
tensor([[ -1, -11,   4,  ...,   0,  -5, -18],
        [ 15,   4, -19,  ...,   9,  10,   8],
        [  4,   0,  13,  ...,   6,   1,   7],
        ...,
        [  8,  10, -11,  ...,  -2,   5,   8],
        [ -3,   4,   1,  ..., -10,  13,  11],
        [-13,  -4,   4,  ...,   7,   5,  -3]], dtype=torch.int32)
torch.Size([10])
tensor([-2192,   134,  -447,   -71,  -915,  1531,  -650,  2332, -3347,   -28],
       dtype=torch.int32)
tensor([[255, 245,   4,  ...,   0, 251, 238],
        [ 15,   4, 237,  ...,   9,  10,   8],
        [  4,   0,  13,  ...,   6,   1,   7],
        ...,
        [  8,  10, 245,  ..., 254,   5,   8],
        [253,   4,   1,  ..., 246,  13,  11],
        [243, 252,   4,  ...,   7,   5, 253]], dtype=torch.int32)
tensor([63344,   134, 65089, 65465, 64621,  1531, 64886,  2332, 62189, 65508],
       dtype=torch.int32)


In [13]:
######################### 신경망 통과 값 추출 ###########################
print(np.shape(model.conv1_out_np))
np.savetxt('out_conv1_value_1.mem', model.conv1_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv1_value_2.mem', model.conv1_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv1_value_3.mem', model.conv1_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.mp1_out_np))
np.savetxt('out_mp1_value_1.mem', model.mp1_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp1_value_2.mem', model.mp1_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp1_value_3.mem', model.mp1_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.conv2_out_np))
np.savetxt('out_conv2_value_1.mem', model.conv2_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv2_value_2.mem', model.conv2_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv2_value_3.mem', model.conv2_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.mp2_out_np))
np.savetxt('out_mp2_value_1.mem', model.mp2_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp2_value_2.mem', model.mp2_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp2_value_3.mem', model.mp2_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.fc_in_np))
np.savetxt('fc_in_value.mem', model.fc_in_np*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.fc_out_np))
np.savetxt('fc_out_value.mem', model.fc_out_np*128, fmt='%1.5d',delimiter = " ")

(1, 3, 24, 24)
(1, 3, 12, 12)
(1, 3, 8, 8)
(1, 3, 4, 4)
(1, 48)
(1, 10)


In [14]:
# 원래 입력 / 가중치 / 바이어스 출력
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)

print("Original Input")
print(model.mp1_out_np[0])
orig_input_1 = model.mp1_out_np[0][0]
orig_input_2 = model.mp1_out_np[0][1]
orig_input_3 = model.mp1_out_np[0][2]

print("Original Weight")
print(model.conv2.weight.detach().numpy())
orig_weight = model.conv2.weight.detach().numpy()

print("Original Bias")
print(model.conv2.bias.detach().numpy())
orig_bias = model.conv2.bias.detach().numpy()

orig_output_calc_1 = np.zeros((3,8,8))
orig_output_calc_2 = np.zeros((3,8,8))
orig_output_calc_3 = np.zeros((3,8,8))
orig_output_calc = np.zeros((3,8,8))

for c in range(3):
    for i in range(8):
        for j in range(8):
            orig_output_calc_1[c][i][j] += (orig_input_1[i:i+5, j:j+5] * orig_weight[c][0]).sum()
            orig_output_calc_2[c][i][j] += (orig_input_2[i:i+5, j:j+5] * orig_weight[c][1]).sum()
            orig_output_calc_3[c][i][j] += (orig_input_3[i:i+5, j:j+5] * orig_weight[c][2]).sum()         
            orig_output_calc[c][i][j] = orig_output_calc_1[c][i][j] + orig_output_calc_2[c][i][j] + orig_output_calc_3[c][i][j] + orig_bias[c]
        
print("\nBias : ")
print(orig_bias)
print("\nCalc Value :\n")
print(orig_output_calc) 
print("\nReal Value :\n")
print(model.conv2_out_np[0])

Original Input
[[[0.007946 0.007946 0.007946 0.007946 0.485303 2.433931 2.86805
   0.795214 0.007946 0.007946 0.007946 0.007946]
  [0.007946 0.007946 0.007946 0.526232 2.294008 4.878462 4.494352
   1.776778 0.269568 0.007946 0.007946 0.007946]
  [0.007946 0.007946 0.262226 2.138685 4.997587 5.42438  5.123318
   3.718413 2.462729 0.392078 0.007946 0.007946]
  [0.007946 0.052655 1.224695 4.752855 5.336329 5.290852 4.306835
   3.687219 3.75194  1.586619 0.145947 0.007946]
  [0.007946 0.087386 2.709602 5.223812 4.915937 3.461112 1.492861
   2.755593 3.816344 3.520272 0.83291  0.007946]
  [0.007946 0.289716 3.459296 4.596095 3.143979 0.517921 0.
   1.717676 4.691989 4.296669 1.12771  0.010092]
  [0.007946 0.49768  4.383057 4.611019 0.754726 0.       0.737615
   3.018578 5.127934 4.624765 0.979043 0.03125 ]
  [0.007946 0.789257 4.539495 4.514042 1.136255 1.673958 3.273121
   5.175742 5.172872 3.462619 0.049458 0.007946]
  [0.007946 0.688944 4.365803 4.677119 3.816629 4.618104 5.225662
   5.2

   -4.26109 ]]]

Real Value :

[[[-0.828657  0.704893  1.951454  4.398038  7.253537  7.254332  2.941462
   -1.957195]
  [ 0.741294  1.855782  4.447139  9.646738 12.662814 11.908775  6.842013
    1.550577]
  [ 0.119945  1.614863  4.51439   7.893129 10.060531 10.734888  8.299784
    4.94633 ]
  [-1.64158   0.571877  4.172332  4.380099  3.920083  4.610086  6.142311
    6.784405]
  [-2.08283   1.332775  4.652186  1.525881 -0.761069  0.436467  3.622389
    7.281419]
  [ 0.116472  5.234763  9.012963  4.75481   0.47695   0.313369  3.093289
    7.89415 ]
  [ 4.854158 12.622501 16.285841  9.297841  4.304612  3.844903  5.803882
    8.757852]
  [ 6.964129 14.440686 16.891525 11.215445  7.455584  7.052281  8.527936
    8.914791]]

 [[ 1.146417  0.586187 -1.169071 -2.942539 -1.610942  2.453888  7.262853
    8.690933]
  [-1.834294 -2.29562  -3.389921 -5.245375 -6.100524 -3.842926  0.442187
    4.9631  ]
  [-1.033363 -2.016812 -2.564297 -2.824316 -2.422157 -2.913472 -2.215978
   -0.722873]
  [ 1.7221

In [15]:
np.set_printoptions(precision=0)

print("Convolution Input")
print(np.shape(model.mp1_out_np))
_input = model.mp1_out_np[0] * 128
print(_input)
print('\n')

print("Convolution Weight")
print(np.shape(model.conv2.weight.detach().numpy()))
weight = model.conv2.weight.detach().numpy() * 128
print(weight)

print("Convolution Bias")
print(np.shape(model.conv2.bias.detach().numpy()))
bias = model.conv2.bias.detach().numpy() * 128
print(bias)

print("Convolution Output")
print(np.shape(model.conv2_out_np))
output = model.conv2_out_np * 128
print(output)

Convolution Input
(1, 3, 12, 12)
[[[  1.   1.   1.   1.  62. 312. 367. 102.   1.   1.   1.   1.]
  [  1.   1.   1.  67. 294. 624. 575. 227.  35.   1.   1.   1.]
  [  1.   1.  34. 274. 640. 694. 656. 476. 315.  50.   1.   1.]
  [  1.   7. 157. 608. 683. 677. 551. 472. 480. 203.  19.   1.]
  [  1.  11. 347. 669. 629. 443. 191. 353. 488. 451. 107.   1.]
  [  1.  37. 443. 588. 402.  66.   0. 220. 601. 550. 144.   1.]
  [  1.  64. 561. 590.  97.   0.  94. 386. 656. 592. 125.   4.]
  [  1. 101. 581. 578. 145. 214. 419. 662. 662. 443.   6.   1.]
  [  1.  88. 559. 599. 489. 591. 669. 677. 603. 136.   0.   1.]
  [  1.  55. 352. 520. 611. 662. 654. 522. 219.   0.   1.   1.]
  [  1.  14. 146. 262. 405. 433. 314. 129.   2.   0.   1.   1.]
  [  1.   1.  19.  87. 114.  73.  31.   0.   1.   1.   1.   1.]]

 [[  0.   0.   0.   0.  69. 304. 365. 213.   0.   0.   0.   0.]
  [  0.   0.   0.  57. 202. 348. 388. 300.  86.   0.   0.   0.]
  [  0.   0.  28. 211. 289. 280. 314. 506. 415. 146.   0.   0.]
  [  

In [16]:
print(np.shape(_input))
print(np.shape(weight))
print(np.shape(output))

_input_1 = _input[0]
_input_2 = _input[1]
_input_3 = _input[2]

output_calc_1 = np.zeros((3,8,8))
output_calc_2 = np.zeros((3,8,8))
output_calc_3 = np.zeros((3,8,8))
output_calc = np.zeros((3,8,8))

for c in range(3):
    for i in range(8):
        for j in range(8):
            output_calc_1[c][i][j] += (_input_1[i:i+5, j:j+5] * weight[c][0]).sum()
            output_calc_2[c][i][j] += (_input_2[i:i+5, j:j+5] * weight[c][1]).sum()
            output_calc_3[c][i][j] += (_input_3[i:i+5, j:j+5] * weight[c][2]).sum()
            output_calc[c][i][j] = output_calc_1[c][i][j] + output_calc_2[c][i][j] + output_calc_3[c][i][j] + bias[c] * 128
        
print("\nBias : ")
print(bias)
print("\nCalc Value :\n")
print(output_calc / 128) 
print("\nReal Value :\n")
print(output)

(3, 12, 12)
(3, 3, 5, 5)
(1, 3, 8, 8)

Bias : 
[-4. -6.  3.]

Calc Value :

[[[ -106.    90.   250.   563.   928.   929.   377.  -251.]
  [   95.   238.   569.  1235.  1621.  1524.   876.   198.]
  [   15.   207.   578.  1010.  1288.  1374.  1062.   633.]
  [ -210.    73.   534.   561.   502.   590.   786.   868.]
  [ -267.   171.   595.   195.   -97.    56.   464.   932.]
  [   15.   670.  1154.   609.    61.    40.   396.  1010.]
  [  621.  1616.  2085.  1190.   551.   492.   743.  1121.]
  [  891.  1848.  2162.  1436.   954.   903.  1092.  1141.]]

 [[  147.    75.  -150.  -377.  -206.   314.   930.  1112.]
  [ -235.  -294.  -434.  -671.  -781.  -492.    57.   635.]
  [ -132.  -258.  -328.  -362.  -310.  -373.  -284.   -93.]
  [  220.   374.   522.   452.   378.   128.    -8.  -228.]
  [  398.   921.  1310.  1186.   798.   411.   135.  -143.]
  [  -34.   426.   980.   984.   575.   167.   -74.  -113.]
  [ -749.  -695.  -445.  -217.  -185.  -292.  -411.  -273.]
  [ -589.  -831. -1024

In [17]:
np.set_printoptions(precision=0)
print("\nCalc Value 1 :\n")
print(output_calc_1) 

print("\nCalc Value 2 :\n")
print(output_calc_2) 

print("\nCalc Value 3 :\n")
print(output_calc_3) 

print("Sum")
print(output_calc_1 + output_calc_2 + output_calc_3)


Calc Value 1 :

[[[ -79563. -115480. -119358.  -83185.  -17614.    8089.  -19178.
    -43569.]
  [ -76744.  -95861.  -57902.   21259.   71909.   58105.    3008.
    -30303.]
  [ -77871.  -72264.    -109.   69870.   77375.   31645.   -8140.
    -15770.]
  [ -86202.  -63898.    8516.   29483.      24.  -51432.  -51033.
     -8752.]
  [ -99805.  -74930.  -25465.  -50144. -102209. -119513.  -67308.
     22131.]
  [ -88203.  -55248.  -29550.  -82149. -126760. -100607.  -18583.
     77434.]
  [ -40519.    7861.   28970.  -29656.  -44251.    1708.   71418.
    120942.]
  [    207.   57557.   83441.   56935.   63897.   92173.  121985.
    118717.]]

 [[ -34046.  -46650.  -68184.  -87613.  -83399.  -52588.   -9879.
     10516.]
  [ -49032.  -62741.  -77445.  -91922.  -91984.  -72616.  -38571.
     -7807.]
  [ -50142.  -56353.  -62881.  -67248.  -65087.  -66195.  -46464.
    -24871.]
  [ -39451.  -32929.  -31157.  -33193.  -32252.  -46073.  -45074.
    -38079.]
  [ -42196.  -27861.  -15240.  -1